# 基于嵌入式文件系统和外部存储的混合文件系统中间件设计

## 1.目的

梳理软件设计，便于程序理解和维护。

## 2.范围 

  适用于乐舜D3110X及同类的网关产品。  

## 3.背景

嵌入式的文件系统通常无法存放较大的文件(0.5GB或以上)。当较大的配置文件需要保存在嵌入式设备中时，一般采用手动存储在外部存储(flash)的方式。这种文件系统和手动存储在外部存储的文件并存的情况会导致接口不统一，可理解、可移植性较差的问题。故引入混合文件系统中间件（以下简称中间件）的设计。

## 4.总体设计

混合文件系统中间件的设计采用面向对象的思想。定义了文件类file_t。

In [ ]:
typedef struct _file_t file_t;
struct _file_t
{
    // 成员函数声明
    const file_backend_t *backend;
    void *backend_data;
};

其中，backend_data是自定义数据指针，下面会详细说明。

根据文件类特性，抽象出文件操作接口。

In [ ]:
typedef struct _file_backend
{
    int32_t 	(*fseek) (file_t *file, int32_t offset, uint8_t opt);				// 定位文件
    int32_t 	(*fread) (file_t *file, uint8_t *buff, uint32_t size); 				// 读文件
    int32_t 	(*fwrite) (file_t *file, uint8_t *buff, uint32_t size); 			// 写文件
    int32_t 	(*fclose) (file_t *file); 											// 关闭文件
} file_backend_t;

## 5.详细设计

### 5.1 对上层（应用层）接口设计

#### 5.1.1 open接口

由于不同存储方式（文件系统、外部flash等）需要的输入参数不同，故初始化文件对象的函数（构造函数）也不相同。对上层（应用层）接口要根据文件名找到文件的存储类型，然后调用对应的构造函数获取文件对象。

文件映射表如下。

In [ ]:
// 文件ID, 存储方式, flash首地址
static int32_t g_config_file_map[LS_CONFIG_FILE_ID_MAX][3] = {
	{LS_CONFIG_FILE_NORTH_BIN, LS_FILE_SAVE_TYPE_EXTERNAL_FLASH, NORTH_BIN_START_ADDR},
	{LS_CONFIG_FILE_SOUTH_BIN, LS_FILE_SAVE_TYPE_EXTERNAL_FLASH, SOUTH_BIN_START_ADDR},
	{LS_CONFIG_FILE_NORTH_LINK_JSON, LS_FILE_SAVE_TYPE_FILE_SYSTEM, 0},
	{LS_CONFIG_FILE_SOUTH_BUS_JSON_1, LS_FILE_SAVE_TYPE_FILE_SYSTEM, 0},
	{LS_CONFIG_FILE_SOUTH_BUS_JSON_2, LS_FILE_SAVE_TYPE_FILE_SYSTEM, 0},
};

open接口如下。

In [ ]:
/*****************************************************************************
* Function	   : OpenFile()
* Description  : 打开文件
* Input 	   : pathname:	带路径的文件名
				 mode: 		打开文件选项
* Output	   : None
* Return	   : file_t *	打开成功，此为该文件句柄
				 NULL		打开失败
* Note(s)	   :
* Contributor  : 2021年6月8日        Andre
*****************************************************************************/
file_t *LS_FileOpen(const int8_t *pathname, uint32_t mode)
{
	int32_t file_id, i;
	file_t *file = NULL;

	if(pathname == NULL)
		return NULL;
	
	file_id = LS_FileGetID(pathname);
	if(file_id < 0)
		return NULL;
	
	//根据存储方式调用不同处理方法
	for(i=0;i<LS_CONFIG_FILE_ID_MAX;i++)
	{
		if(file_id == g_config_file_map[i][0])
		{
			if(g_config_file_map[i][1] == LS_FILE_SAVE_TYPE_FILE_SYSTEM)
				file = LS_FileOpenFibo(pathname, mode); 			
			else if(g_config_file_map[i][1] == LS_FILE_SAVE_TYPE_EXTERNAL_FLASH)
				file = LS_FileOpenFlash(pathname, mode, g_config_file_map[i][2]);
			else
				Log_e("file %s save type:%d error!", pathname, g_config_file_map[i][1]);
		}
	}
	
	return file;
}

#### 5.1.2 seek/read/write/close接口

这四个接口直接调用文件对象自身的对应方法即可。以read为例。

In [ ]:
/*****************************************************************************
* Function	   : LS_FileRead()
* Description  : 从打开文件的当前位置读出数据
* Input 	   : file_t:	文件对象
				 buff: 		数据缓冲区指针
 				 size:		数据缓冲区长度
* Output	   : None
* Return	   : >=0 		读出的字节数
				 <0 		失败
* Note(s)	   :
* Contributor  : 2021年7月2日        Andre
*****************************************************************************/
int32_t LS_FileRead(file_t *file, uint8_t *buff, uint32_t size)
{
	if(file == NULL)
		return -1;
	
	if(buff == NULL)
		return -1;
	
	return file->backend->fread(file, buff, size);
}

#### 5.1.3 exist/getsize接口

这两个接口的实现方式同open接口。没有在文件操作接口中集成的原因是：通常会在初始化文件对象的函数（open函数）之前调用这两个方法，而不是之后。所以这两个方法不属于文件对象file_t。这也是沿袭了大多数文件系统的做法。

这里只展示这两个函数头文件。

In [ ]:
/*****************************************************************************
* Function	   : LS_FileExist()
* Description  : 检测文件是否存在
* Input 	   : pathname:	带路径的文件名
* Output	   : None
* Return	   : 1 		文件存在
				 <0 	文件不存在
* Note(s)	   :
* Contributor  : 2021年7月2日        Andre
*****************************************************************************/

In [ ]:
/*****************************************************************************
* Function	   : LS_FileGetSize()
* Description  : 获取文件大小
* Input 	   : pathname:	带路径的文件名
* Output	   : None
* Return	   : >=0 	文件长度
				 <0 	操作失败
* Note(s)	   :
* Contributor  : 2021年7月2日        Andre
*****************************************************************************/

### 5.2 中间件对接原有文件系统设计

由于各文件系统之间大同小异,此处以广和通L610嵌入式文件系统为例。

#### 5.2.1 成员函数和自定义数据指针

对于基于原有文件系统的文件对象，文件对象的成员函数就是原有文件系统的对应方法（函数）。

In [ ]:
const file_backend_t _file_system_fibo_backend = {
	_LS_FileSeekFibo,
	_LS_FileReadFibo,
	_LS_FileWriteFibo,
	_LS_FileCloseFibo
};

文件对象的自定义数据指针指向用于存放文件描述符的地址空间。其内容如下。

In [ ]:
typedef struct _file_fibo
{
    int32_t fd;				//广和通文件系统文件句柄
} file_fibo_t;

#### 5.2.2 初始化文件对象的函数（构造函数）

由于混合文件系统中间件大多数的对上接口设计都是参考标准文件系统设计。所以构造函数只需要调用原有文件系统的构造函数,即open函数。并把原有文件系统的seek/read/write/close函数通过结构体数组的方式绑定到新申请文件对象的成员函数指针上。自定义数据指针指向用于存放文件描述符的地址空间。

In [ ]:
/*****************************************************************************
* Function	   : LS_FileOpenFibo()
* Description  : 打开文件-基于广和通文件系统
* Input 	   : pathname:	带路径的文件名
				 mode: 		打开文件选项
* Output	   : None
* Return	   : file_t *	打开（创建）成功，此为该文件句柄
				 NULL		创建失败
* Note(s)	   :
* Contributor  : 2021年7月1日        Andre
*****************************************************************************/
file_t *LS_FileOpenFibo(const int8_t *pathname, uint32_t mode)
{
    file_t *file;
	file_fibo_t *file_fibo;

    file = (file_t *)fibo_malloc(sizeof(file_t));
	if(file == NULL)
	{
		Log_e("malloc fail");
		return NULL;
	}

	file->backend = &_file_system_fibo_backend;
    file->backend_data = (file_fibo_t *)fibo_malloc(sizeof(file_fibo_t));
	if(file->backend_data == NULL)
	{
		Log_e("malloc fail");
		if(file != NULL) 
			fibo_free((void *)file);
		return NULL;
	}
	file_fibo = (file_fibo_t *)file->backend_data;
	file_fibo->fd = fibo_file_open(pathname, mode);
	if(file_fibo->fd < 0)
	{
		Log_e("open file %s failed!", pathname);
		if(file->backend_data != NULL) 
			fibo_free((void *)file->backend_data);
		if(file != NULL) 
			fibo_free((void *)file);
		return NULL;		
	}

	return file;
}

#### 5.2.3 seek/read/write/close函数

直接调用原有文件系统的对应函数。以read为例。

In [ ]:
/*****************************************************************************
* Function	   : _LS_FileReadFibo()
* Description  : 从打开文件的当前位置读出数据
* Input 	   : file_t:	文件对象
				 buff: 		数据缓冲区指针
 				 size:		数据缓冲区长度
* Output	   : None
* Return	   : >=0 		读出的字节数
				 <0 		失败
* Note(s)	   :
* Contributor  : 2021年7月2日        Andre
*****************************************************************************/
static int32_t _LS_FileReadFibo(file_t *file, uint8_t *buff, uint32_t size)
{
    int32_t i, ret;
	file_fibo_t *file_fibo_data = file->backend_data;

	for(i=0;i<size/FIBO_FILESYSTEM_MAX_READ_WRITE_SIZE;i++)
	{
		ret = fibo_file_read(file_fibo_data->fd, buff+(i*FIBO_FILESYSTEM_MAX_READ_WRITE_SIZE), FIBO_FILESYSTEM_MAX_READ_WRITE_SIZE);
		if(ret < 0)
		{
			Log_e("fibo_file_read fail!");
			return -1;
		}
	}
	if(size%FIBO_FILESYSTEM_MAX_READ_WRITE_SIZE != 0)
	{
		ret = fibo_file_read(file_fibo_data->fd, buff+(i*FIBO_FILESYSTEM_MAX_READ_WRITE_SIZE), size%FIBO_FILESYSTEM_MAX_READ_WRITE_SIZE);
		if(ret < 0)
		{
			Log_e("fibo_file_read fail!");
			return -1;
		}
	}
	
	return size;
}

#### 5.2.4 exist/getsize函数

直接调用原有文件系统的对应函数。以exist为例。

In [ ]:
/*****************************************************************************
* Function	   : LS_FileExistFibo()
* Description  : 检测文件是否存在-基于广和通文件系统
* Input 	   : pathname:	带路径的文件名
* Output	   : None
* Return	   : 1 		文件存在
				 <0 	文件不存在
* Note(s)	   :
* Contributor  : 2021年7月1日        Andre
*****************************************************************************/
int32_t LS_FileExistFibo(const int8_t *pathname)
{
	return fibo_file_exist(pathname);
}

### 5.3 中间件对接外部存储设计

这里以外部flash为例。外部flash相关驱动已具备。如果是其他存储介质，可以参照此设计进行移植。

#### 5.3.1 成员函数和自定义数据指针

对于基于外部flash的文件对象，文件对象的成员函数是根据文件系统的原理设计的一系列外部flash操作集。

In [ ]:
const file_backend_t _file_system_flash_backend = {
	_LS_FileSeekFlash,
	_LS_FileReadFlash,
	_LS_FileWriteFlash,
	_LS_FileCloseFlash
};

文件对象的自定义数据指针指向的地址空间包括文件首地址，读写指针等相关参数。

In [ ]:
typedef struct _file_flash
{
    uint32_t	file_save_address;						// 文件首地址
    uint32_t 	file_pointer_offset;					// 文件读写指针
    uint32_t 	file_size;								// 文件长度    
} file_flash_t;

#### 5.3.2 初始化文件对象的函数（构造函数）

基于外部flash文件对象的构造函数和基于原有文件系统文件对象的构造函数有相似之处。主要是把自己设计的seek/read/write/close函数通过结构体数组的方式绑定到新申请文件对象的成员函数指针上。自定义数据指针指向用于存放文件首地址，读写指针等相关参数的地址空间。

In [ ]:
/*****************************************************************************
* Function	   : LS_FileOpenFlash()
* Description  : 打开文件-基于外部flash
* Input 	   : pathname:		文件名
				 mode: 			打开文件选项
				 save_address:	文件首地址
* Output	   : None
* Return	   : file_t *	打开（创建）成功，此为该文件句柄
				 NULL		创建失败
* Note(s)	   :
* Contributor  : 2021年7月2日        Andre
*****************************************************************************/
file_t *LS_FileOpenFlash(const int8_t *pathname, uint32_t mode, uint32_t save_address)
{
    int32_t ret;
	file_t *file;
	file_flash_t *file_flash;

    file = (file_t *)fibo_malloc(sizeof(file_t));
	if(file == NULL)
	{
		Log_e("fibo_malloc fail");
		return NULL;
	}

	file->backend = &_file_system_flash_backend;
    file->backend_data = (file_flash_t *)fibo_malloc(sizeof(file_flash_t));
	if(file->backend_data == NULL)
	{
		Log_e("fibo_malloc fail");
		if(file != NULL) 
			fibo_free((void *)file);
		return NULL;
	}
	file_flash = (file_flash_t *)file->backend_data;
	file_flash->file_save_address = save_address;
	file_flash->file_pointer_offset = 0;		// 打开时指针默认在文件头
	ret = LS_FileGetSizeFlash(pathname, save_address);
	if(ret < 0)
	{
		Log_e("LS_FileGetSizeFlash %s failed!", pathname);
		if(file->backend_data != NULL) 
			fibo_free((void *)file->backend_data);
		if(file != NULL) 
			fibo_free((void *)file);
		return NULL;		
	}
	else
		file_flash->file_size = ret;

	return file;
}

#### 5.2.3 seek/read/write/close函数

seek函数是把自定义数据中的读写指针移动到相应的位置。

In [ ]:
/*****************************************************************************
* Function	   : _LS_FileSeekFlash()
* Description  : 移动文件数据指针，并返回操作成功后的文件指针位置
* Input 	   : file_t:	文件对象
				 offset: 	从文件开始的偏移
 				 opt:		偏移的起始位置
* Output	   : None
* Return	   : >=0 		返回操作成功后文件指针位置
				 <0 		失败
* Note(s)	   :
* Contributor  : 2021年7月2日        Andre
*****************************************************************************/
static int32_t _LS_FileSeekFlash(file_t *file, int32_t offset, uint8_t opt)
{
    file_flash_t *file_flash_data = file->backend_data;

	switch(opt)
	{
		case FS_SEEK_SET:
			file_flash_data->file_pointer_offset = offset;
			break;
		case FS_SEEK_CUR:
			file_flash_data->file_pointer_offset += offset;
			break;
		case FS_SEEK_END:
			file_flash_data->file_pointer_offset = file_flash_data->file_size+offset;
			break;
		default:
			return -1;
			break;
	}
//	Log_d("file_flash_data->file_pointer_offset is %d!", file_flash_data->file_pointer_offset);
	
	if(file_flash_data->file_pointer_offset > file_flash_data->file_size)
		return -1;
	else
		return file_flash_data->file_pointer_offset;
}

read函数是在读写指针的位置开始读取指定的字节数，并把读写指针移动到结束的位置。注意，如果指定的字节数超过外部flash驱动单次可读取的最大字节数，要分段进行读取。

In [ ]:
/*****************************************************************************
* Function	   : _LS_FileReadFlash()
* Description  : 从打开文件的当前位置读出数据
* Input 	   : file_t:	文件对象
				 buff: 		数据缓冲区指针
 				 size:		数据缓冲区长度
* Output	   : None
* Return	   : >=0 		读出的字节数
				 <0 		失败
* Note(s)	   :
* Contributor  : 2021年7月2日        Andre
*****************************************************************************/
static int32_t _LS_FileReadFlash(file_t *file, uint8_t *buff, uint32_t size)
{
    int32_t i, ret;
	file_flash_t *file_flash_data = file->backend_data;

//	Log_d("file_flash_data->file_save_address is %d!", file_flash_data->file_save_address);
//	Log_d("file_flash_data->file_pointer_offset is %d!", file_flash_data->file_pointer_offset);
//	Log_d("file_flash_data->file_size is %d!", file_flash_data->file_size);

	if(file_flash_data->file_pointer_offset + size > file_flash_data->file_size)
	{
		Log_e("read over file_size fail!");
		return -1;
	}

	for(i=0;i<size/FLASH_EX_MAX_READ_WRITE_SIZE;i++)
	{
		ret = LS_flash_ex_read(file_flash_data->file_save_address+file_flash_data->file_pointer_offset, buff+(i*FLASH_EX_MAX_READ_WRITE_SIZE), FLASH_EX_MAX_READ_WRITE_SIZE);
		if(ret < 0)
		{
			Log_e("LS_flash_ex_read fail!");
			return -1;
		}
		else
			file_flash_data->file_pointer_offset += FLASH_EX_MAX_READ_WRITE_SIZE;
	}
	if(size%FLASH_EX_MAX_READ_WRITE_SIZE != 0)
	{
		ret = LS_flash_ex_read(file_flash_data->file_save_address+file_flash_data->file_pointer_offset, buff+(i*FLASH_EX_MAX_READ_WRITE_SIZE), size%FLASH_EX_MAX_READ_WRITE_SIZE);
		if(ret < 0)
		{
			Log_e("fibo_file_read fail!");
			return -1;
		}
		else
			file_flash_data->file_pointer_offset += size%FLASH_EX_MAX_READ_WRITE_SIZE;
	}
	
	return size;
}

write和close函数同理。

#### 5.2.4 exist/getsize函数

exist函数通过读取文件中的关键字段来确认文件是否存在。同理，getsize函数也是通过读取关键字段（如结构体个数）来计算文件大小的。这里以getsize函数为例。

In [ ]:
/*****************************************************************************
* Function	   : LS_FileGetSizeFlash()
* Description  : 获取文件大小-基于外部flash
* Input 	   : pathname:	带路径的文件名
				 save_address:  文件首地址
* Output	   : None
* Return	   : >=0	文件长度
				 <0 	操作失败
* Note(s)	   :
* Contributor  : 2021年7月1日		Andre
*****************************************************************************/
int32_t LS_FileGetSizeFlash(const int8_t *pathname, uint32_t save_address)
{
	int32_t i, ret;
	uint8_t slave_num = 0, point_num_temp[DTU_RS485_NUM*MAX_METER_NUM*2] = {0};
	uint16_t point_num[DTU_RS485_NUM*MAX_METER_NUM] = {0};
	uint32_t flash_address_index = 0, file_size = 0;

	// 读从设备总数
	ret = LS_flash_ex_read(save_address, &slave_num, 1);
	if(ret < 0)
	{
		Log_e("LS_flash_ex_read error!");
		return -1;
	}
//	Log_d("slave_num is %d!", slave_num);
	
	if(slave_num > DTU_RS485_NUM*MAX_METER_NUM)
	{
		Log_e("slave_num error!");
		return -1;
	}
	flash_address_index = 1;

	// 读单个从设备测点数
#if ONE_BYTE_CODE_LEN
	ret = LS_flash_ex_read(save_address+flash_address_index, point_num_temp, slave_num);
	if(ret < 0)
	{
		Log_e("LS_flash_ex_read error!");
		return -1;
	}

	for(i=0;i<slave_num;i++)
		point_num[i] = point_num_temp[i];
	flash_address_index += slave_num;	
#else
	ret = LS_flash_ex_read(save_address+flash_address_index, point_num_temp, slave_num*2);
	if(ret < 0)
	{
		Log_e("LS_flash_ex_read error!");
		return -1;
	}
	// 大小端转换
	for(i=0;i<slave_num;i++)
		point_num[i] = (point_num_temp[i*2]<<8)+point_num_temp[i*2+1];
	flash_address_index += slave_num*2; 
#endif

	for(i=0;i<slave_num;i++)
	{
		if(point_num[i] > DTU_VM_MAX)
		{
			Log_e("node[%d]: %d is too large, error!", i, point_num[i]);
			return -1;
		}	
	}

	//计算文件大小
    if(strstr(pathname,NORTH_BIN) != NULL)
	{
		file_size = slave_num*sizeof(config_file_server_code_ied_t)+ sizeof(config_file_server_code_app_t) + flash_address_index;
		for(i=0;i<slave_num;i++)
		{
			file_size += (sizeof(config_file_server_code_code_t) * point_num[i]);
//			Log_d("sizeof(config_file_server_code_code_t) is %d", sizeof(config_file_server_code_code_t));
//			Log_d("point_num[%d] is %d", i, point_num[i]);
		}
	}
	else if(strstr(pathname,SOUTH_BIN) != NULL)
	{
		file_size = slave_num*sizeof(config_file_codes_template_ied_t)+ sizeof(config_file_codes_template_app_t) + flash_address_index;
		for(i=0;i<slave_num;i++)
		{
			file_size += (sizeof(config_file_codes_template_code_t) * point_num[i]);
//			Log_d("sizeof(config_file_server_code_code_t) is %d", sizeof(config_file_codes_template_code_t));
//			Log_d("point_num[%d] is %d", i, point_num[i]);
		}
	}
	else
	{
        Log_e("LS_FileGetSizeFlash %s error", pathname);
		return -1;			
    }
//	Log_d("bin file size is :%d",file_size);

	return file_size;
}

## 6.不足和改进方法

1.文件类的抽象接口的方法只包含文件的基本操作，需要后续进一步扩展。

2.基于外部flash的文件对象的exist/getsize函数设计不够严谨。后续可考虑flash中增加文件信息头（包括校验码，文件是否存在标志，文件大小，文件在外部flash中的存储地址等），用作检验；而不是用关键字段来进行优化。

3.依赖广和通头文件，后续可使用自定义宏来减少对广和通头文件的依赖。

4.指针操作频繁。要注意malloc和free的对应，防止内存泄露。